In [ ]:
import lpips
import torch
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

Configuring directories

In [ ]:
distorted_images_dir = "Distortions_v2"
csv_output_dir = "CSVs"
chart_output_dir = "Charts"

Measuring values and saving to csv file

In [ ]:
# Initialize LPIPS model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_fn = lpips.LPIPS(net='alex').to(device)

# Transform: convert image to tensor and normalize to [-1, 1]
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Optional: adjust to match your data
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # LPIPS expects input in [-1, 1]
])

results = []

# Loop over picture folders (Degus, Hotdog, etc.)
for scene_folder in os.listdir(distorted_images_dir):
    scene_path = os.path.join(distorted_images_dir, scene_folder)
    if not os.path.isdir(scene_path):
        continue

    # Loop over distortion type folders (Blur, Noise, etc.)
    for distortion_folder in os.listdir(scene_path):
        distortion_path = os.path.join(scene_path, distortion_folder)
        if not os.path.isdir(distortion_path):
            continue

        # Find ref image (ends with 0.bmp and is only .bmp file)
        reference_image = None
        for file in os.listdir(distortion_path):
            if file.endswith('_0.bmp'):
                reference_image = os.path.join(distortion_path, file)
                break
        if reference_image is None:
            print(f"No reference image found in {scene_path}")
            continue
        ref_img = transform(Image.open(reference_image).convert('RGB')).unsqueeze(0).to(device)

        # Calculate lpips for each picture
        for file in os.listdir(distortion_path):
            if not file.endswith('.bmp') and not file.endswith('.png'):
                continue

            distorted_image_path = os.path.join(distortion_path, file)

            # Load and preprocess distorted image
            dist_img = transform(Image.open(distorted_image_path).convert('RGB')).unsqueeze(0).to(device)

            # Compute LPIPS distance
            with torch.no_grad():
                distance = loss_fn(ref_img, dist_img).item()

            #extract distortion value
            distortion_value = -1
            match = re.search(r'^([^_]+)_([^_]+)_(.*)\.(png|bmp)$', file)
            try:
                distortion_value = float(match.group(3))
            except:
                print(f"No distortion value found in {file}")

            # Save result
            results.append({
                'Picture': scene_folder,
                'DistortionType': distortion_folder,
                'DistortionValue': distortion_value,
                'Filename': file,
                'LPIPS': distance
            })

# Save all results to CSV
df = pd.DataFrame(results)
os.makedirs(csv_output_dir, exist_ok=True)
df.to_csv(csv_output_dir + '/lpips_distortion_results.csv', index=False)
print("✅ Results saved to lpips_distortion_results.csv")

Creating charts

In [ ]:
# === Load CSV ===
df = pd.read_csv(csv_output_dir + "/lpips_distortion_results.csv")

# Add synthetic LPIPS=0 for level 0 reference images
reference_rows = []
for picture in df['Picture'].unique():
    for distortion in df['DistortionType'].unique():
        reference_rows.append({
            'Picture': picture,
            'DistortionType': distortion,
            'DistortionValue': 0,
            'Filename': 'image_0.bmp',
            'LPIPS': 0.0
        })

df = pd.concat([df, pd.DataFrame(reference_rows)], ignore_index=True)

# === Plotting ===
os.makedirs(chart_output_dir, exist_ok=True)

for distortion in df['DistortionType'].unique():
    plt.figure(figsize=(8, 5))
    distortion_df = df[df['DistortionType'] == distortion]

    for picture in sorted(distortion_df['Picture'].unique()):
        picture_df = distortion_df[distortion_df['Picture'] == picture]
        picture_df = picture_df.sort_values('DistortionValue')

        # Build y and x values
        x = picture_df['DistortionValue']
        y = picture_df['LPIPS']
        plt.plot(x, y, marker='o', label=picture)

    plt.title(f'LPIPS vs Distortion Value – {distortion}')
    plt.xlabel('Distortion Parameter')
    plt.ylabel('LPIPS Distance')
    plt.legend(title='Picture', fontsize='small')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{chart_output_dir}/{distortion}_lpips_chart.png')
    plt.close()

print("✅ All charts saved")

Creating group chart

In [ ]:
# # === Load CSV and filter for _4.bmp ===
# df = pd.read_csv(csv_output_dir + "/lpips_distortion_results.csv")
# df = df[df['Filename'].str.endswith('_4.bmp')]
# df['Level'] = 4
#
# # === Parse level 4 descriptions from markdown ===
# def parse_md_level_4_descriptions(md_path):
#     with open(md_path, 'r') as f:
#         lines = f.readlines()
#
#     descriptions = {}
#     current_type = None
#     for line in lines:
#         line = line.strip()
#         if line.startswith("##"):
#             header = line[2:].strip()
#             match = re.match(r'\d+\.\s*([^\(]+)', header)
#             if match:
#                 current_type = match.group(1).strip()
#         elif line.startswith("- Level 4:") and current_type:
#             desc = line.split(":", 1)[1].strip()
#             descriptions[current_type] = desc
#     return descriptions
#
# desc_mapping = parse_md_level_4_descriptions("Image distorion Description.md")
#
# # === Clean distortion names and attach descriptions ===
# df['DistortionTypeClean'] = df['DistortionType'].str.strip()
# df['Description'] = df['DistortionTypeClean'].map(desc_mapping)
# df['Label'] = df.apply(
#     lambda row: f"{row['DistortionTypeClean']} ({row['Description']})" if pd.notnull(row['Description']) else row['DistortionTypeClean'],
#     axis=1
# )
#
# # === Get unique distortions and scenes ===
# distortions = sorted(df['Label'].unique())
# scenes = sorted(df['Scene'].unique())
#
# # === Prepare grouped bar chart data ===
# bar_width = 0.8 / len(scenes)  # fit all scene bars per group
# x = np.arange(len(distortions))  # X positions for each group
#
# colors = plt.cm.tab10.colors
# fig, ax = plt.subplots(figsize=(14, 6))
#
# # Plot bars per scene
# for i, picture in enumerate(scenes):
#     heights = []
#     for dist in distortions:
#         match = df[(df['Label'] == dist) & (df['Scene'] == picture)]
#         heights.append(match['LPIPS'].values[0] if not match.empty else 0)
#
#     ax.bar(x + i * bar_width, heights, bar_width, label=picture, color=colors[i % len(colors)])
#
# # === Final plot formatting ===
# ax.set_xticks(x + bar_width * (len(scenes) - 1) / 2)
# ax.set_xticklabels(distortions, rotation=45, ha='right')
# ax.set_ylabel("LPIPS Distance")
# ax.set_title("LPIPS Scores at Distortion Level 4 (Grouped by Distortion Type)")
# ax.legend(title="Scene")
# plt.tight_layout()
# plt.savefig("charts/bar_chart_level4_grouped_by_distortion.png")
# plt.show()
